In [26]:
# from the provided file 'US_Presidents_unordered.txt':
    # seperate the data into the following in order:
        # first_name
        # middle_intial
        # last_name
        # begin_term
        # end_term
        # presedential order

    # Create a report as a new txt file containing the following in presidential order:
        # order_number
        # first_name 
        # middle_initial
        # last_name
        # begin_term
        # end_term 
        # years_in_office

    # Save the data into a database into the respective columns in presidential order:
        # order text,  
        # first_name text, 
        # middle_initial text,
        # last_name text 
        # begin_term text, 
        # end_term text, 
        # years_in_office real,

In [101]:
import sqlite3

data = {}
with open("US_Presidents_unordered.txt", "r") as file:
    line = file.readline()
    while line:
        # Skip first line in file
        if 'Name,Term_Start,Term_End' not in line: 
            # Filter / , ' . characters out of the string
            line = line.replace('\'','').replace('.','').replace(',',' ')

            # Change the string into a list of items seperated by spaces
            line = line.split()

            # If there is no middle name
            if len(line) == 5:

                # Insert n/a as the middle name
                line.insert(2, ' ')

            # Store presidents years in office at the end of the list
            # (Using the difference of the end/start dates)
            line.append(str(int(line[5][-4:]) - int(line[4][-4:])))

            # store data into a dictionary with the presidents order number as a key
            data[int(line[0])] = line
        line = file.readline()

# Sort dictionary by it's items index, then convert back to a dictionary
data = dict(sorted(data.items()))

In [102]:
# Open the file in write mode (will be created if it doesn't exist)
file = open("solution.txt", 'w')
# Format the list to a writeable string
[file.write(' '.join(data[line])+'\n') for line in data]
file.close()

In [105]:
# Save that data to the database
connection = sqlite3.connect('united_states.db')
cursor = connection.cursor()
cursor.execute('''create table if not exists presidents (
        order_number text, 
        first_name text, 
        middle_initial text, 
        last_name text, 
        begin_term text, 
        end_term text, 
        years_in_office real
)''')
[cursor.execute('insert into presidents values(?,?,?,?,?,?,?)', data[item]) for item in data]
connection.commit()
connection.close()

In [ ]:
# Retreive presidents from the database in presidential order
connection = sqlite3.connect('united_states.db')
cursor = connection.cursor()
presidents = cursor.execute('select * from presidents')
[print(president) for president in presidents]
connection.close()

In [158]:
# Query all presidents whose last name begins with 'R' or 'A'
connection = sqlite3.connect('united_states.db')
cursor = connection.cursor()
presidents = list(cursor.execute("select * from presidents where last_name like 'R%' or last_name like 'A%'"))
connection.close()
[print(president)for president in presidents]

('2', 'John', ' ', 'Adams', '03/04/1797', '03/04/1801', 4.0)
('6', 'John', 'Quincy', 'Adams', '03/04/1825', '03/04/1829', 4.0)
('21', 'Chester', ' ', 'Arthur', '09/19/1881', '03/04/1885', 4.0)
('26', 'Theodore', ' ', 'Roosevelt', '09/14/1901', '03/04/1909', 8.0)
('32', 'Franklin', 'D', 'Roosevelt', '03/04/1933', '04/12/1945', 12.0)
('40', 'Ronald', ' ', 'Reagan', '01/20/1981', '01/20/1989', 8.0)


[None, None, None, None, None, None]

In [157]:
# Get a count of all the letters and how often they occur
# Smashing all of the names into a string
massive_string = ''.join([''.join(data[president][1:4]) for president in data]).lower().replace(' ','')
# key:value pairs of letters and how often they occur
results = dict(sorted({letter : massive_string.count(letter) for letter in massive_string}.items()))
print(results)

{'a': 57, 'b': 12, 'c': 16, 'd': 23, 'e': 55, 'f': 10, 'g': 17, 'h': 29, 'i': 32, 'j': 15, 'k': 8, 'l': 35, 'm': 23, 'n': 53, 'o': 49, 'p': 2, 'q': 1, 'r': 59, 's': 29, 't': 19, 'u': 9, 'v': 9, 'w': 14, 'x': 1, 'y': 12, 'z': 1}


In [152]:
# Number of presidents in a century
term_dates = [(data[president][4][-4:-2],data[president][5][-4:-2]) for president in data]
results = {'1700s':0, '1800s':0, '1900s':0, '2000s':0}

# Checking centuries against the presidents active years
for term in term_dates:
    for century in results:
        if century[:2] in term:
            results[century] += 1
print(results)

{'1700s': 2, '1800s': 24, '1900s': 18, '2000s': 3}
